# Extraction Phase

In [14]:
#Extract the data 

import json , csv
from number_parser import parse
from data_exploring import DataExploration

# load movies_raw file

def load_json_file(file_location):
    with open(file_location , 'r') as file:
        file_content = json.load(file)

    return file_content


def  load_csv_as_listOfDict(file_location , delim = ','):
    data = []
    with open(file_location ,'r' , newline='') as file:
        content = csv.DictReader(file , delimiter=delim)
        for row in content:
            data.append(row)

    return data


movies = load_json_file('./movies_raw.json')
users  = load_csv_as_listOfDict('./users_raw.csv')  
ratings  = load_csv_as_listOfDict('./ratings_raw.csv')  

    

In [15]:
#print(movies[0])
print(users[0])
print(type(ratings[0]['rating_id']))

{'user_id': '1001', 'name': 'Erika', 'age': '14', 'country': 'EG'}
<class 'str'>


# Transformation Phase

In [16]:
# Exploring Movies file

#DataExploration.explore(users , 5)
#DataExploration.explore(ratings , 5)

# Exploring the data types

DataExploration.explore_dict(users[0])
DataExploration.explore_dict(ratings[0])
DataExploration.explore_dict(movies[0])



user_id : <class 'str'>
name : <class 'str'>
age : <class 'str'>
country : <class 'str'>
rating_id : <class 'str'>
user_id : <class 'str'>
movie_id : <class 'str'>
rating : <class 'str'>
timestamp : <class 'str'>
id : <class 'int'>
title : <class 'str'>
genre : <class 'str'>
release_year : <class 'str'>


In [17]:
# Data Checking 

users_stats = DataExploration.checking_missing_values(users)
ratings_stats = DataExploration.checking_missing_values(ratings)
movies_stats = DataExploration.checking_missing_values(movies)


print(users_stats)
print(ratings_stats)
print(movies_stats)



{'missing_age': 63, 'missing_country': 16}
{'missing_timestamp': 354, 'missing_rating': 178}
{'missing_release_year': 62, 'missing_genre': 24}


In [18]:

# Data Cleaning for users data
cleaned_users   = DataExploration.data_cleaning(users)
cleaned_movies  = DataExploration.data_cleaning(movies)  
cleaned_ratings = DataExploration.data_cleaning(ratings)


print(DataExploration.checking_missing_values(cleaned_users))
print(DataExploration.checking_missing_values(cleaned_movies))
print(DataExploration.checking_missing_values(cleaned_ratings))



{}
{}
{}


# Converting Invalid Data Types

In [19]:

# Converting data  types of users data

for user in cleaned_users:
    user['user_id'] = int(user['user_id'])
    user['age'] = int(user['age'])

#print(cleaned_users[0])

In [20]:
#Converting Data types of movies data

for movie in cleaned_movies:
    
    # Now year is cleaned, convert and assign
    movie['release_year'] = DataExploration.parse_english_to_numbers(str(movie['release_year']).lower())  # Work on a local variable


#cleaned_movies

In [21]:


for rate in cleaned_ratings:
    rate['rating'] = DataExploration.parse_english_to_numbers(str(rate['rating']).lower())



# Drop any record have invalid rating t must be from (0 to 5)
temp = []
for rate in cleaned_ratings:
    if  0<= rate['rating'] <= 5:
        temp.append(rate)

#print(len(cleaned_ratings))
#cleaned_ratings = temp
#print(len(cleaned_ratings))

#cleaned_ratings

### Enhanced User / Movie Dict 

In [22]:


def identify_dict_records(dictList , dict_key):
    
    """
        accept list of dicts and transform it to dict it's key is the dict id
        
    """

    temp = {}
    #print(key , type(key))
    for record in dictList:
        dict_record = {}
        dict_record[record[str(dict_key)]] = record
        temp[record[str(dict_key)]] = dict_record

    return temp


#Dict of users , movies
cleaned_users = identify_dict_records(cleaned_users , list(cleaned_users[0].keys())[0])  
cleaned_movies = identify_dict_records(cleaned_movies ,list(cleaned_movies[0].keys())[0])



cleaned_users


{1001: {1001: {'user_id': 1001, 'name': 'Erika', 'age': 14, 'country': 'EG'}},
 1003: {1003: {'user_id': 1003, 'name': 'Scott', 'age': 62, 'country': 'USA'}},
 1007: {1007: {'user_id': 1007,
   'name': 'Robert',
   'age': 28,
   'country': 'Germany'}},
 1010: {1010: {'user_id': 1010,
   'name': 'Cassandra',
   'age': 40,
   'country': 'EG'}},
 1012: {1012: {'user_id': 1012,
   'name': 'Brett',
   'age': 42,
   'country': 'France'}},
 1013: {1013: {'user_id': 1013,
   'name': 'Timothy',
   'age': 28,
   'country': 'USA'}},
 1019: {1019: {'user_id': 1019,
   'name': 'Christopher',
   'age': 70,
   'country': 'Germany'}},
 1024: {1024: {'user_id': 1024,
   'name': 'Christopher',
   'age': 53,
   'country': 'egypt'}},
 1025: {1025: {'user_id': 1025,
   'name': 'Renee',
   'age': 41,
   'country': 'Germany'}},
 1026: {1026: {'user_id': 1026,
   'name': 'Lisa',
   'age': 31,
   'country': 'egypt'}},
 1028: {1028: {'user_id': 1028, 'name': 'Sarah', 'age': 18, 'country': 'EG'}},
 1033: {1033: 

In [23]:
type(cleaned_users)


dict

In [24]:
"""
    * Rating Details table *

    username , user_age , user_county , movie_title , movie_genre , rating , date_of_rate , time_of_rate 
"""


rating_details = []

for rate in cleaned_ratings:
    
    # Check user forign Key exist in users data & Movie id also

    
    if cleaned_users.get(int(rate['user_id'])) is None or cleaned_movies.get(int(rate['movie_id'])) is None:
        continue

    rating_record = {}
    user_details = cleaned_users[int(rate['user_id'])][int(rate['user_id'])]
    movie_details = cleaned_movies[int(rate['movie_id'])][int(rate['movie_id'])]
    rating_record['username'] = user_details['name']
    rating_record['user_age'] = user_details['age']
    rating_record['user_country'] = user_details['country']
    rating_record['movie_title'] = movie_details['title']
    rating_record['movie_genre'] = movie_details['genre']
    rating_record['rating'] = rate['rating']
    
    rating_details.append(rating_record)


list(rating_details[0].keys())



['username',
 'user_age',
 'user_country',
 'movie_title',
 'movie_genre',
 'rating']

# Load Phase into SQL Database

In [25]:

from database_connection import DatabaseConnection

DatabaseConnection.write_dictList_to_database(rating_details)



TypeError: DatabaseConnection.write_dictList_to_database() missing 1 required positional argument: 'dictList'